# Week 5 Homework

***Due (pushed to your GitHub branch) on 10/18 by 11:59 pm***

## FASTQ fetch, QC, and trimming/filtering

Use the SRA-toolkit to fetch the relevant FASTQ files. You may use a hand-crafted `txt` file to provide the list of SRAs instead of manipulating an SRA Run Table with `cut`.

In [1]:
!mkdir fastq
!cut -d ',' -f 1 SraRunTableB.txt | tail -n +2 > sra_listB.txt
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --sra-id $line --threads 32 --outdir FASTQ_B --gzip; \
    done < sra_listB.txt

mkdir: cannot create directory ‘fastq’: File exists
Getting SRR1790682 from NCBI SRA
2024-10-17 17:59:19,425 - SRR ids: ['SRR1790682']
2024-10-17 17:59:19,425 - extra args: ['--gzip']
2024-10-17 17:59:19,425 - tempdir: /local/scratch/job_99900/pfd_v1m2z533
2024-10-17 17:59:19,425 - CMD: sra-stat --meta --quick SRR1790682
2024-10-17 17:59:21,302 - SRR1790682 spots: 26205429
2024-10-17 17:59:21,302 - blocks: [[1, 818919], [818920, 1637838], [1637839, 2456757], [2456758, 3275676], [3275677, 4094595], [4094596, 4913514], [4913515, 5732433], [5732434, 6551352], [6551353, 7370271], [7370272, 8189190], [8189191, 9008109], [9008110, 9827028], [9827029, 10645947], [10645948, 11464866], [11464867, 12283785], [12283786, 13102704], [13102705, 13921623], [13921624, 14740542], [14740543, 15559461], [15559462, 16378380], [16378381, 17197299], [17197300, 18016218], [18016219, 18835137], [18835138, 19654056], [19654057, 20472975], [20472976, 21291894], [21291895, 22110813], [22110814, 22929732], [22929

Run QC on the FASTQ files and aggregate the results into a report.

In [3]:
!mkdir FASTQ_B/qc
!fastqc -t 32 FASTQ_B/*.fastq.gz -o FASTQ_B/qc
!multiqc --force -d FASTQ_B/qc/

mkdir: cannot create directory ‘FASTQ_B/qc’: File exists
application/gzip
application/gzip
Started analysis of SRR1790682.fastq.gz
application/gzip
application/gzip
Started analysis of SRR1790683.fastq.gz
application/gzip
application/gzip
Started analysis of SRR1790684.fastq.gz
application/gzip
Started analysis of SRR1790685.fastq.gz
application/gzip
application/gzip
application/gzip
Started analysis of SRR1790686.fastq.gz
Started analysis of SRR1790687.fastq.gz
Started analysis of SRR1790688.fastq.gz
Started analysis of SRR1790689.fastq.gz
Started analysis of SRR1790690.fastq.gz
Started analysis of SRR1790691.fastq.gz
Approx 5% complete for SRR1790691.fastq.gz
Approx 10% complete for SRR1790691.fastq.gz
Approx 15% complete for SRR1790691.fastq.gz
Approx 5% complete for SRR1790689.fastq.gz
Approx 20% complete for SRR1790691.fastq.gz
Approx 25% complete for SRR1790691.fastq.gz
Approx 30% complete for SRR1790691.fastq.gz
Approx 5% complete for SRR1790682.fastq.gz
Approx 5% complete for S

Edit this block with a short evaluation of each of the report sections. Each evaluation should include your important observations, reasons for potential QC failure, and an opinion on what should be done on the failure (i.e., trimming or filtering). Problem FASTQ files, if any, should be noted.

*Per base sequence quality: Overall quite good no red at all.

*Per tile sequence quality: Also quite good all green with the exception of a few yellows.

*Per sequence quality scores: Overall quite good no red at all.

*Per base sequence content: Not great There is a lot of red and yellow not so much green. Mostly because of the duplicates that are in the sequencing.

*Per sequence GC content: Very middle of the pack some green and some red mostly yellow. This is because of a very high GC content and can be fixed my filtering the set.

*Per base N content: Overall quite good no red at all.

*Sequence length distribution: Overall quite good no red at all.

*Sequence duplication levels: Lots of red and green because of multiple duplicates. We need to get rid of the duplicates.

*Overrepresented sequences: Almost all sequences are overrepresented we can fix this by trimming them down.

*Adapter content: Vewry good almost all green with very few yellows.

If necessary, use `cutadapt` to trim and filter your reads. Provide a justification for the `cutadapt` parameters you used:



Run QC on the trimmed/filtered reads and generate an aggregated report

In [1]:
%mkdir trimmed_B
!for fastq in FASTQ_B/*.fastq.gz; do \
    base_name=$(basename "$fastq" .fastq.gz); \
    cutadapt -j 32 -u 10 -o ./trimmed_B/$base_name.fastq.gz $fastq; \
done

mkdir: cannot create directory ‘trimmed_B’: File exists
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -u 10 -o ./trimmed_B/SRR1790682.fastq.gz FASTQ_B/SRR1790682.fastq.gz
Processing single-end reads on 32 cores ...
Done           00:00:59    26,205,429 reads @   2.3 µs/read;  26.46 M reads/minute
Finished in 59.516 s (2.271 µs/read; 26.42 M reads/minute).

=== Summary ===

Total reads processed:              26,205,429
Reads written (passing filters):    26,205,429 (100.0%)

Total basepairs processed: 4,716,977,220 bp
Total written (filtered):  4,454,922,930 bp (94.4%)
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -u 10 -o ./trimmed_B/SRR1790683.fastq.gz FASTQ_B/SRR1790683.fastq.gz
Processing single-end reads on 32 cores ...
Done           00:01:00    26,126,780 reads @   2.3 µs/read;  25.93 M reads/minute
Finished in 60.534 s (2.317 µs/read; 25.90 M reads/minute).

=== Summary ===

Total reads processed:              26,126,780
Reads w

In [2]:
!mkdir trimmed_B/qc
!fastqc -t 32 trimmed_B/*.fastq.gz -o trimmed_B/qc
!multiqc --force -d FASTQ_B/qc/ trimmed_B/qc/

mkdir: cannot create directory ‘trimmed_B/qc’: File exists
application/gzip
application/gzip
Started analysis of SRR1790682.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of SRR1790683.fastq.gz
application/gzip
application/gzip
application/gzip
Started analysis of SRR1790684.fastq.gz
application/gzip
Started analysis of SRR1790685.fastq.gz
Started analysis of SRR1790686.fastq.gz
Started analysis of SRR1790687.fastq.gz
Started analysis of SRR1790688.fastq.gz
Started analysis of SRR1790689.fastq.gz
Started analysis of SRR1790690.fastq.gz
Started analysis of SRR1790691.fastq.gz
Approx 5% complete for SRR1790691.fastq.gz
Approx 10% complete for SRR1790691.fastq.gz
Approx 15% complete for SRR1790691.fastq.gz
Approx 5% complete for SRR1790689.fastq.gz
Approx 20% complete for SRR1790691.fastq.gz
Approx 25% complete for SRR1790691.fastq.gz
Approx 5% complete for SRR1790682.fastq.gz
Approx 5% complete for SRR1790683.fastq.gz
Approx 30% complete for

Report your observations on which QC metrics improved (or got worse): All of the QC metrics got better this was most of them just needed trimming due to the large number of duplicates found in the sequencing.

## Snakemake Pipeline

When running bioinformatics studies, it is wise to use notebooks (such as this one) to tie together code, decisions, and data visualization. The above task is something you might generate and report to a client or supervisor who might be interested in sequencing QC and what kind of pre-processing decisions you made prior to alignment. In addition, the preprocessing should be readily reproducible by all relevant stakeholders, so you should supplement notebooks with pipeline scripts. Throughout the semester, we will be creating snakemake pipelines to reproduce the published analysis that you've selected. Follow the below steps to begin producing this pipeline:

1. Make a new `Snakefile` and `config.yaml` in `5_fastq`. 
2. In the `Snakefile`, produce the following rules:  
    a. `fetch_fastq` - use `parallel-fastq-dump` to fetch every FASTQ file listed in `config.yaml`.  
    b. `fastq_qc` - use `fastqc` to run QC on the raw FASTQ files.  
    c. `trim_filter` - use `cutadapt` to trim and/or filter the FASTQ reads using parameters justified in this notebook. ***Note:*** if you choose to trim/filter, it should be done on *all* of the FASTQ files; do not cherry pick.  
    d. `trim_qc` - use `fastqc` to run QC on the trimmed FASTQ files.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`. You may use the documentation found [here](https://multiqc.info/docs/usage/pipelines/#snakemake) or any other online documentation you may find. You may not use any AI tools to complete this homework.